In [18]:
#Setup

# Install Packages
import sys
!{sys.executable} -m pip install numpy --quiet
!{sys.executable} -m pip install matplotlib --quiet
!{sys.executable} -m pip install seaborn --quiet
!{sys.executable} -m pip install sklearn --quiet
!{sys.executable} -m pip install torch --quiet

from sklearn import preprocessing
import os, requests
from matplotlib import rcParams 
from matplotlib import pyplot as plt
import numpy as np
import seaborn as sns
import pandas as pd
from sklearn.decomposition import PCA
from sklearn import datasets, linear_model
from sklearn.metrics import mean_squared_error, r2_score
import math
from sklearn.linear_model import LinearRegression
import torch
from torch.autograd import Variable
import torch.nn as nn

# Data Retrieval

fname = 'joystick_track.npz'
url = "https://osf.io/6jncm/download"

print('Is cuda?', torch.cuda.is_available())

if not os.path.isfile(fname):
  try:
    r = requests.get(url)
  except requests.ConnectionError:
    print("!!! Failed to download data !!!")
  else:
    if r.status_code != requests.codes.ok:
      print("!!! Failed to download data !!!")
    else:
      with open(fname, "wb") as fid:
        fid.write(r.content)
# Import matplotlib and set styling
rcParams['figure.figsize'] = [20, 4]
rcParams['font.size'] =15
rcParams['axes.spines.top'] = False
rcParams['axes.spines.right'] = False
rcParams['figure.autolayout'] = True

colourmap_diverge = sns.diverging_palette(321, 172, s=100, n=100, center = "light", as_cmap=True)
colourmap = sns.color_palette("rocket", as_cmap=True)
colourmap = sns.light_palette("#30887c", as_cmap=True)
colourmap_diverge.set_bad("black", alpha=0)
colourmap.set_bad("black", alpha=0)

# Data Loading
alldat = np.load(fname, allow_pickle=True)['dat']

# Select just one of the recordings here. This is subject 1, block 1.
dat = alldat[0, 3]

Is cuda? False


In [19]:
print('Is cuda?', torch.cuda.is_available())
if torch.cuda.is_available():
    !nvcc --version
    torch.set_default_tensor_type('torch.cuda.FloatTensor')
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


torch.manual_seed(3)


Is cuda? False


In [20]:
# Define normalisation of voltage channels

Vmax =  max((max(x) for x in dat['V']))
Vmin = min((min(x) for x in dat['V']))
xmax = max(dat['cursorX'])
xmin = min(dat['cursorX'])
ymax = max(dat['cursorY'])
ymin = min(dat['cursorY'])

def Vnormalise(_V):
    _V_norm = (_V - Vmin)/(Vmax - Vmin)
    return _V_norm

def normalise(_V):
    _V_norm = (_V - min(_V))/(max(_V) - min(_V))
    return _V_norm

def Xnormalise(_x):
    _x_norm = (_x - xmin)/(xmax - xmin)
    return _x_norm
    
def Xdenormalise(_x):
    _x_denorm = _x*(xmax - xmin) + xmin
    return _x_denorm

def Ynormalise(_y):
    _y_norm = (_y - ymin)/(ymax - ymin)
    return _y_norm
    
def Ydenormalise(_y):
    _y_denorm = _y*(ymax - ymin) + ymin
    return _y_denorm

  
def downsample(signal, factor):
  nbins = math.floor(nt/factor)
  signal_norm = np.zeros((nbins, 1))
  for ibin in range(nbins):
    binstart = ibin * 40
    binend = binstart + 40
    signal_norm[ibin, 0] = np.mean(signal[binstart:binend])

  return signal_norm


In [21]:
ds = True #downsample data?
select_channels = False #Use correlated channels only?


# Load patient 2
dat = alldat[0, 2]
corr_chan = sorted([5, 23, 17, 1, 25])

V = dat['V']
V = Vnormalise(V)
nt, nchan = V.shape

if select_channels:
  V = V[:, corr_chan]

cx = Xnormalise(dat['cursorX'].flatten()).reshape(-1,1)
cy = Ynormalise(dat['cursorY'].flatten()).reshape(-1,1)

print('number of samples', nt, 'number of channels', nchan)
print('V shape', V.shape)
print('cx, cy shape', cx.shape, cy.shape)

factor = 40 #cursor trajectory updates every 40 samples, hardware limitation
if downsample:
  nbins = math.floor(nt/factor)

  V_norm = np.zeros((nbins, nchan))
  for c in range(nchan):
    V_norm[:,c] = downsample(V[:,c], 40).flatten()
  V = V_norm
  cx = downsample(cx, 40)
  cy = downsample(cy, 40)

  print("post normalisation:")
  print('number of samples', nt, 'number of channels', nchan)
  print('V shape', V.shape)
  print('cx, cy shape', cx.shape, cy.shape)



number of samples 134360 number of channels 64
V shape (134360, 64)
cx, cy shape (134360, 1) (134360, 1)
post normalisation:
number of samples 134360 number of channels 64
V shape (3359, 64)
cx, cy shape (3359, 1) (3359, 1)


In [22]:
def split_sequences(input_sequences, output_sequence, n_steps_in, n_steps_out):
    X, y = list(), list() # instantiate X and y
    for i in range(len(input_sequences)):
        # find the end of the input, output sequence
        end_ix = i + n_steps_in
        out_end_ix = end_ix + n_steps_out - 1
        # check if we are beyond the dataset
        if out_end_ix > len(input_sequences): break
        # gather input and output of the pattern
        seq_x, seq_y = input_sequences[i:end_ix], output_sequence[end_ix-1:out_end_ix, -1]
        X.append(seq_x), y.append(seq_y)
    return np.array(X), np.array(y)

In [23]:
def generate_tensors(X, y, tt_ratio = 0.8, lookback = 200, lookahead = 25):
    X_ss, y_ss = split_sequences(X, y, lookback, lookahead)

    total_samples = len(X)
    train_test_cutoff = round(tt_ratio* total_samples)

    X_train = X_ss[:train_test_cutoff]
    X_test = X_ss[train_test_cutoff:]

    y_train = y_ss[:train_test_cutoff]
    y_test = y_ss[train_test_cutoff:] 

    X_train_tensors = Variable(torch.Tensor(X_train))
    X_test_tensors = Variable(torch.Tensor(X_test))

    y_train_tensors = Variable(torch.Tensor(y_train))
    y_test_tensors = Variable(torch.Tensor(y_test))

    X_train_tensors_final = torch.reshape(X_train_tensors,   
                                          (X_train_tensors.shape[0], lookback, 
                                           X_train_tensors.shape[2]))
    X_test_tensors_final = torch.reshape(X_test_tensors,  
                                         (X_test_tensors.shape[0], lookback, 
                                          X_test_tensors.shape[2])) 

    print("Training Shape:", X_train_tensors_final.shape, y_train_tensors.shape)
    print("Testing Shape:", X_test_tensors_final.shape, y_test_tensors.shape) 
    print("Test/Train cutoff", train_test_cutoff)
    print("Total data points", total_samples)
    
    return X_train_tensors_final, y_train_tensors, X_test_tensors_final, y_test_tensors




In [24]:
class LSTM(nn.Module):
    
    def __init__(self, num_classes, input_size, hidden_size, num_layers):
        super().__init__()
        self.num_classes = num_classes # output size
        self.num_layers = num_layers # number of recurrent layers in the lstm
        self.input_size = input_size # input size
        self.hidden_size = hidden_size # neurons in each lstm layer
        # LSTM model
        self.lstm = nn.LSTM(input_size=input_size, hidden_size=hidden_size,
                            num_layers=num_layers, batch_first=True, dropout=0.2) # lstm
        self.fc_1 =  nn.Linear(hidden_size, 100) # fully connected 
        self.fc_2 = nn.Linear(100, num_classes) # fully connected last layer
        self.relu = nn.ReLU()
        
    def forward(self,x):
        # hidden state
        h_0 = Variable(torch.zeros(self.num_layers, x.size(0), self.hidden_size))
        # cell state
        c_0 = Variable(torch.zeros(self.num_layers, x.size(0), self.hidden_size))
        # propagate input through LSTM
        output, (hn, cn) = self.lstm(x, (h_0, c_0)) # (input, hidden, and internal state)
        hn = hn.view(-1, self.hidden_size) # reshaping the data for Dense layer next
        out = self.relu(hn)
        out = self.fc_1(out) # first dense
        out = self.relu(out) # relu
        out = self.fc_2(out) # final output
        return out

In [25]:
def training_loop(n_epochs, lstm, optimiser, loss_fn, X_train, y_train,
                  X_test, y_test):
    test_losses = []
    train_losses = []
    for epoch in range(n_epochs):
        lstm.train()
        outputs = lstm.forward(X_train) # forward pass
        optimiser.zero_grad() # calculate the gradient, manually setting to 0
        # obtain the loss function
        loss = loss_fn(outputs, y_train)
        loss.backward() # calculates the loss of the loss function
        optimiser.step() # improve from loss, i.e backprop
        # test loss
        lstm.eval()
        test_preds = lstm(X_test)
        test_loss = loss_fn(test_preds, y_test)
        train_losses.append(loss.item())
        test_losses.append(test_loss.item())
        if epoch % 100 == 0 or (epoch < 50 and epoch % 10 == 0):
            print("Epoch: %d, train loss: %1.5f, test loss: %1.5f" % (epoch, 
                                                                      loss.item(), 
                                                                      test_loss.item())) 
    return train_losses, test_losses

In [26]:
# Define plotting functions
def predict(model, inputs, targets, tt_ratio, test=False):
  df_X_ss = inputs
  df_y_mm = targets
  # split the sequence
  X_ss, y_ss = split_sequences(inputs, targets, lookback, lookahead)

  # converting to tensors
  df_X_ss = Variable(torch.Tensor(X_ss))
  df_y_mm = Variable(torch.Tensor(y_ss))
  # reshaping the dataset
  df_X_ss = torch.reshape(df_X_ss, (df_X_ss.shape[0], lookback, df_X_ss.shape[2]))

  train_predict = model(df_X_ss) # forward pass
  data_predict = train_predict.cpu().data.numpy() # numpy conversion
  dataY_plot = df_y_mm.cpu().data.numpy()


  true, preds = [], []
  for i in range(len(dataY_plot)):
      true.append(dataY_plot[i][0])
  for i in range(len(data_predict)):
      preds.append(data_predict[i][0])

  total_samples = len(true)
  train_test_cutoff = round(tt_ratio* total_samples)

  print("Predicted trajectory length:", len(true))
  if test:
    true = true[train_test_cutoff:]
    preds = preds[train_test_cutoff:]
    print("Predicted trajectory TEST length:", len(true))
  
  
  return true, preds

def plot_prediction(true, preds, tt_ratio):

  total_samples = len(true)
  train_test_cutoff = round(tt_ratio* total_samples)

  plt.figure(figsize=(20,6)) #plotting
  plt.subplot(1, 2, 1)
  plt.plot(true, label='Actual Data') # actual plot
  plt.plot(preds, label='Predicted Data') # predicted plot
  plt.axvline(x=train_test_cutoff, c='r', linestyle='--') # size of the training set
  plt.title('Time-Series Prediction')
  plt.legend()

  plt.subplot(1, 2, 2)
  plt.plot(true, label='Actual Data') # actual plot
  plt.plot(preds, label='Predicted Data') # predicted plot
  plt.title('Time-Series Prediction')
  plt.legend()
  plt.savefig("whole_plot.png", dpi=300)
  plt.xlim(left = train_test_cutoff - train_test_cutoff*0/1)
  plt.axvline(x=train_test_cutoff, c='r', linestyle='--') # size of the training set
  plt.show() 

  print("test/train boundary:", train_test_cutoff)
  print("test/train proportion:", train_test_cutoff/total_samples)
  print("train length:", total_samples - train_test_cutoff)
  rms_test = rms(np.array(true), np.array(preds))
  print(rms_test)


def plot_losses(train_loss, test_loss):
  plt.plot(train_loss, label="Train Losses")
  plt.plot(test_loss, label="Test Losses")
  plt.legend

def bin_trajectory(x, y, nbins):
  bin_x = np.array_split(x, nbins)
  bin_y = np.array_split(y, nbins)
  bin_traj = []
  
  for i in range(len(bin_x)):
    traj_chunk = np.asarray([bin_x[i], bin_y[i]])
    bin_traj.append(traj_chunk)

  binsize = traj_chunk.shape[1]
  return bin_traj, binsize

def plot_trajectories(true_x, true_y, preds_x, preds_y, nbins = 5, shadow = False):
  true_traj_bin, binsize = bin_trajectory(true_x, true_y, nbins=nbins)
  pred_traj_bin, binsize = bin_trajectory(preds_x, preds_y, nbins=nbins)

  print(int(binsize/4))

  h = 3
  plt.figure(figsize = (h*nbins, h))
  for i, b in enumerate(range(nbins)):
    plt.subplot(1, nbins, b+1)
    if shadow:
      for j in [-1, 1]:
        try:
          plt.plot(true_traj_bin[i+j][0][int(binsize/4):], true_traj_bin[i+j][1][int(binsize/4):], color='gainsboro')
          plt.plot(pred_traj_bin[i+j][0][:int(binsize/4)], pred_traj_bin[i+j][1][:int(binsize/4)], color='gainsboro')
        except:
          pass
      
    plt.plot(true_traj_bin[i][0], true_traj_bin[i][1], label="true trajectory")
    plt.plot(pred_traj_bin[i][0], pred_traj_bin[i][1], label="predicted trajectory")

  plt.legend()


In [27]:
#Define common metric between models
def rms(truth, prediction):
    _diff = truth - prediction
    _diff_flat = _diff.flatten()
    _rms = np.sqrt(np.mean(_diff_flat**2))
    return _rms

In [28]:
# data preprocessing parameters
train_test_proportion = 0.8
lookback, lookahead = 400, 200 

In [29]:
# model architecture parameters
input_size = nchan # number of features
hidden_size = 10 # number of features in hidden state
num_layers = 1 # number of stacked lstm layers

num_classes = lookahead # number of output classes 

Training Shape: torch.Size([2687, 200, 64]) torch.Size([2687, 25])
Testing Shape: torch.Size([449, 200, 64]) torch.Size([449, 25])
Test/Train cutoff 2687
Total data points 3359
Epoch: 0, train loss: 0.48003, test loss: 0.61849
Epoch: 10, train loss: 0.31547, test loss: 0.40488
Epoch: 20, train loss: 0.18048, test loss: 0.21172
Epoch: 30, train loss: 0.11756, test loss: 0.09497
Epoch: 40, train loss: 0.10418, test loss: 0.06177
Epoch: 100, train loss: 0.09929, test loss: 0.07215
Epoch: 200, train loss: 0.09491, test loss: 0.06335
Epoch: 300, train loss: 0.06643, test loss: 0.06373
Epoch: 400, train loss: 0.05817, test loss: 0.06910
Epoch: 500, train loss: 0.05385, test loss: 0.07788
Epoch: 600, train loss: 0.04935, test loss: 0.07798
Epoch: 700, train loss: 0.04418, test loss: 0.08029
Epoch: 800, train loss: 0.04130, test loss: 0.09319
Epoch: 900, train loss: 0.03516, test loss: 0.08489


TypeError: plot_prediction() takes 3 positional arguments but 4 were given

In [ ]:
# learning hyperparameters
import warnings
warnings.filterwarnings('ignore')

n_epochs = 1000 # 2000 epochs
learning_rate = 0.001 # 0.005 lr

In [ ]:
# generate tensors for x model
X_train_tensors_final, y_train_tensors, X_test_tensors_final, y_test_tensors = generate_tensors(V, cx, train_test_proportion, lookback, lookahead)

# instantiate x model
lstm_x = LSTM(num_classes, 
              input_size, 
              hidden_size, 
              num_layers)

# define training methods
loss_fn = torch.nn.MSELoss()    # mean-squared error for regression
optimiser = torch.optim.Adam(lstm_x.parameters(), lr=learning_rate) 

# train model
train_loss_x, test_loss_x = training_loop(n_epochs=n_epochs,
              lstm=lstm_x,
              optimiser=optimiser,
              loss_fn=loss_fn,
              X_train=X_train_tensors_final,
              y_train=y_train_tensors,
              X_test=X_test_tensors_final,
              y_test=y_test_tensors)

# generate final prediction
true_x, preds_x = predict(lstm_x, V, cx, train_test_proportion)


In [ ]:
# generate tensors for y model
X_train_tensors_final, y_train_tensors, X_test_tensors_final, y_test_tensors = generate_tensors(V, cy, train_test_proportion, lookback, lookahead)

# instantiate model
lstm_y = LSTM(num_classes, 
              input_size, 
              hidden_size, 
              num_layers)

# define training methods
loss_fn = torch.nn.MSELoss()    # mean-squared error for regression
optimiser = torch.optim.Adam(lstm_y.parameters(), lr=learning_rate) 

# train model
train_loss_y, test_loss_y = training_loop(n_epochs=n_epochs,
              lstm=lstm_y,
              optimiser=optimiser,
              loss_fn=loss_fn,
              X_train=X_train_tensors_final,
              y_train=y_train_tensors,
              X_test=X_test_tensors_final,
              y_test=y_test_tensors)

# generate final prediction
true_y, preds_y = predict(lstm_y, V, cy, train_test_proportion)

In [ ]:
plot_prediction(true_x, preds_x, train_test_proportion)
plot_losses(train_loss_x, test_loss_x)

plot_prediction(true_y, preds_y, train_test_proportion)
plot_losses(train_loss_y, test_loss_y)

true_x_test, preds_x_test = predict(lstm_x, V, cx, train_test_proportion, test=True)
true_y_test, preds_y_test = predict(lstm_y, V, cy, train_test_proportion, test=True)
plot_trajectories(true_x_test, true_y_test, preds_x_test, preds_y_test, nbins=5, shadow=True)

